In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

In [2]:
# read it in to inspect it
with open('./datasets/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# let's look at the first 100 characters
print(text[:102])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You a


In [5]:
def tokenize(text):
    vocab={'<SOS>':0,'<EOS>':1,'<UNK>':2,'<PAD>':3}
    inv_vocab={0:'<SOS>',1:'<EOS>',2:'<UNK>',3:'<PAD>'}
    for i in range(len(text)):
        sent=text[i].split()
        for s in sent:
            if s in vocab:
                continue
            else:
                vocab[s]=len(vocab)
                inv_vocab[vocab[s]]=s
    return vocab,inv_vocab

In [6]:
sent=text.split('\n')
vocab,inv_vocab=tokenize(sent)

In [8]:
maxlen=0
for i in sent:
    maxlen=max(maxlen,len(i))
print(maxlen)

63


In [9]:
vocab

{'<SOS>': 0,
 '<EOS>': 1,
 '<UNK>': 2,
 '<PAD>': 3,
 'First': 4,
 'Citizen:': 5,
 'Before': 6,
 'we': 7,
 'proceed': 8,
 'any': 9,
 'further,': 10,
 'hear': 11,
 'me': 12,
 'speak.': 13,
 'All:': 14,
 'Speak,': 15,
 'You': 16,
 'are': 17,
 'all': 18,
 'resolved': 19,
 'rather': 20,
 'to': 21,
 'die': 22,
 'than': 23,
 'famish?': 24,
 'Resolved.': 25,
 'resolved.': 26,
 'First,': 27,
 'you': 28,
 'know': 29,
 'Caius': 30,
 'Marcius': 31,
 'is': 32,
 'chief': 33,
 'enemy': 34,
 'the': 35,
 'people.': 36,
 'We': 37,
 "know't,": 38,
 "know't.": 39,
 'Let': 40,
 'us': 41,
 'kill': 42,
 'him,': 43,
 'and': 44,
 "we'll": 45,
 'have': 46,
 'corn': 47,
 'at': 48,
 'our': 49,
 'own': 50,
 'price.': 51,
 "Is't": 52,
 'a': 53,
 'verdict?': 54,
 'No': 55,
 'more': 56,
 'talking': 57,
 "on't;": 58,
 'let': 59,
 'it': 60,
 'be': 61,
 'done:': 62,
 'away,': 63,
 'away!': 64,
 'Second': 65,
 'One': 66,
 'word,': 67,
 'good': 68,
 'citizens.': 69,
 'accounted': 70,
 'poor': 71,
 'citizens,': 72,
 'patri

In [10]:
def pad_tokenize(sent,maxlen,vocab):
    data=[]
    for i in range(len(sent)):
        words=sent[i].split()
        temp=[vocab[j] for j in words]
        if len(temp)<maxlen:
            temp.extend([vocab['<PAD>']]*(maxlen-len(temp)))
        assert len(temp)==maxlen, print('short of maxlen')
        data.extend(temp)
    return data
    
data=pad_tokenize(sent,maxlen,vocab)

In [11]:
vocab_size=len(vocab)

In [12]:
dataset=torch.tensor(data,dtype=torch.long)

In [12]:
dataset[:100]

tensor([ 4,  5,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  6,  7,  8,  9, 10, 11, 12, 13,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3])

In [13]:
n = int(0.9*len(text)) # first 90% will be train, rest val
train_data = dataset[:n]
val_data = dataset[n:]

In [25]:
# hyperparameters
batch_size = 8 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 20000
eval_interval = 100
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embd = 512
n_head = 8
n_layer = 4
dropout = 0.0

In [15]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(f"ix : {ix}")
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [16]:
x_sample,y_sample=get_batch('train')

In [17]:
x_sample

tensor([[    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3],
        [    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3],
        [    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,   306,  4611, 12744,
           747,   122],
        [    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,  

In [18]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [18]:
# print(text[:64])
# x_sample_item,y_sample_item=train.__getitem__(8)
# print(f"X_sample : {x_sample_item}")
# print(f"y_sample : {y_sample_item}")

# print(f"X_sample.shape :{x_sample_item.shape}")

In [19]:
len(val_data)

1516209

In [20]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class wordLevelLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        #print(idx.shape)
        B,T = idx.shape
        #print(f"B : {B} C:{T}")
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [26]:
model = wordLevelLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

38.93665 M parameters
step 0: train loss 10.0240, val loss 10.0226
step 100: train loss 0.7944, val loss 0.7858
step 200: train loss 0.7383, val loss 0.7115
step 300: train loss 0.7638, val loss 0.6894
step 400: train loss 0.7125, val loss 0.7392
step 500: train loss 0.7152, val loss 0.6987
step 600: train loss 0.7543, val loss 0.6790
step 700: train loss 0.6507, val loss 0.7075
step 800: train loss 0.7215, val loss 0.7416
step 900: train loss 0.7394, val loss 0.7364
step 1000: train loss 0.6711, val loss 0.7069
step 1100: train loss 0.7091, val loss 0.7615
step 1200: train loss 0.6885, val loss 0.7658
step 1300: train loss 0.6725, val loss 0.7198
step 1400: train loss 0.6560, val loss 0.6938
step 1500: train loss 0.6591, val loss 0.7257
step 1600: train loss 0.6706, val loss 0.7439
step 1700: train loss 0.6407, val loss 0.6603
step 1800: train loss 0.7024, val loss 0.7148
step 1900: train loss 0.6601, val loss 0.7297
step 2000: train loss 0.6923, val loss 0.7472
step 2100: train loss 

In [27]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
output_tokens=m.generate(context, max_new_tokens=2000)[0].tolist()
res=''
for i in output_tokens:
  if inv_vocab[i]=='<PAD>':
    continue
  res+=inv_vocab[i]
  res+=' '
print(res)

<SOS> A thousand fearful company, not your cause GREGORY: Ah, my lord, the first Twice of countenance. The very trick stood possess'd. If hold our cares and be your name, After you: did need pieces me from his grace. Why shall so, for whom I am not for himself His answer death. He would show us lusty, the repulse of him stand Better I will not king: COMINIUS: And no way but conceit, my soul recorded But who shall conduct CORIOLANUS: His deserts, but of thy growth. Indeed this the do, have guilty thy blameful EXTON: her send him little. What is it is look'd for whom with course, Let not have beware our herein, Cast by their Forthwith once, for I were none. And he were struck it from him: Your Spoke even in hand: The end of bold Come. for than to make God that hath your shall uncle Rivers up I'll betide him run of a thousand men 
